# Twitter sentiment analysis

#### Importing necessary libraries

In [5]:
import os
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

#### Setting Twitter API keys and tokens as environment variables in OS.

    os.environ['TWITTER_CONSUMER_KEY'] = 'your-consumer-key'
    os.environ['TWITTER_CONSUMER_SECRET'] = 'your-consumer-secret'
    os.environ['TWITTER_ACCESS_TOKEN'] = 'your-access-token'
    os.environ['TWITTER_ACCESS_TOKEN_SECRET'] = 'your-access-token-secret'

#### Fetch Twitter API keys and tokens from environment variables

In [ ]:
consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
access_token = os.getenv('TWITTER_ACCESS_TOKEN')
access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')

# Ensure that all keys are retrieved, else raise an error
if not all([consumer_key, consumer_secret, access_token, access_token_secret]):
    raise ValueError("Missing one or more Twitter API keys. Check your environment variables.")

#### Defining the TwitterClient class for sentiment analysis

In [7]:
class TwitterClient(object):

    def __init__(self):
        try:
            # OAuthHandler object for authentication
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth)
        except Exception as e:
            print(f"Error: Authentication Failed - {e}")

    # Utility function to clean tweet text by removing links and special characters using regular expressions.
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    # Function to classify sentiment of a tweet using textblob's sentiment method
    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(self.clean_ttweet)
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    # Function to fetch tweets from the API and parse them.
    def get_tweets(self, query, count=10):
        tweets = []
        try:
            fetched_tweets = self.api.search(q=query, count=count)
            for tweet in fetched_tweets:
                parsed_tweet = {}
                parsed_tweet['text'] = tweet.text
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
            return tweets
        except tweepy.TweepError as e:
            print(f"Error: {e}")

#### Main function to perform sentiment analysis on a sample query

In [8]:
def main():
    api = TwitterClient()

    # Fetch tweets for a query (e.g., 'Donald Trump')
    tweets = api.get_tweets(query='Donald Trump', count=200)

    # Extract positive, negative, and neutral tweets
    positive_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    negative_tweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']

    # Calculate and print the percentage of each sentiment
    print(f"Positive tweets percentage: {100 * len(positive_tweets) / len(tweets):.2f} %")
    print(f"Negative tweets percentage: {100 * len(negative_tweets) / len(tweets):.2f} %")
    print(f"Neutral tweets percentage: {100 * (len(tweets) - len(positive_tweets) - len(negative_tweets)) / len(tweets):.2f} %")

    print("\nPositive tweets:")
    for tweet in positive_tweets[:10]:
        print(tweet['text'])

    print("\nNegative tweets:")
    for tweet in negative_tweets[:10]:
        print(tweet['text'])

In [ ]:
if __name__ == "__main__":
    main()